# Python PowerShell-Remoting

Open an powershell remote session with python to the computer and run the code in the scriptblock.

Press Run all to execute all tests.

### Remote computer to connect to

In [ ]:
computer  = 'TINU0034545.domain.com'

### Scriptblock to execute on the remote computer, this read the following properties:

- <span style="color: rgb(206, 145, 120);">ComputerName</span>
- <span style="color: #ce9178;">LastBootupTime</span>
- <span style="color: #ce9178;">TotalMemoryMB</span>

In [ ]:
# Remote ScriptBlock to execute

ps_script = """
function Get-PyInstalledMemory{
    [CmdletBinding()]
    param(
        [Parameter(Mandatory=$false)]
        [String] $Name
    )
    try{
        $ComputerSystem = Get-WmiObject Win32_ComputerSystem
        $ComputerName   = [System.Net.Dns]::GetHostByName($env:computername).HostName
        $LastBootupTime = (Get-CimInstance Win32_OperatingSystem).LastBootupTime
        $TotalMemoryMB  = [Math]::Round($ComputerSystem.TotalPhysicalMemory/1mb)
    }catch{
        $error.clear()        
    }

    # Return as Json-String for python
    [PSCustomObject]@{
        ComputerName   = $ComputerName
        LastBootupTime = $LastBootupTime
        TotalMemoryMB  = $TotalMemoryMB
    } | Convertto-Json

}
Get-PyInstalledMemory
"""


### Invoke PowerShell Remote Script

In [ ]:
# Invoke PowerShell Remote Script

import winrm, getpass, json, datetime

def invoke_remotescript(computer,user,password,ps_script):
    try:
        session = winrm.Session(computer, auth=(user, password), transport='ntlm')
        result  = session.run_ps(ps_script)
        if result.status_code == 0:
            return result.std_out.decode('utf-8').replace('\r\n','') 
        else:
            return result.std_err

    except Exception as e:
        print(e)

user      = 'scis-taawama9@swisscom.com'
password  = getpass.getpass(prompt='Enter the password for '+str(user), stream=None)

scriptret = invoke_remotescript(computer,user,password,ps_script)
json_object = json.dumps(scriptret, indent = 4)   
print(json_object)
